# Regression Week 5: LASSO (coordinate descent)

In [5]:
import pandas as pd
import numpy as np
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

In [3]:
sales = pd.read_csv('kc_house_data.csv',dtype=dtype_dict)
sales.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340.0,5650.0
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690.0,7639.0
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720.0,8062.0
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360.0,5000.0
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800.0,7503.0


In [4]:
sales['floors'] = sales['floors'].astype(float).astype(int)

In [6]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.as_matrix()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.as_matrix()
    return(feature_matrix, output_array)

In [7]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix,weights)
    return(predictions)

In [8]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix,axis = 0)
    normalized_features = feature_matrix/norms
    return normalized_features,norms

In [9]:
features, norms = normalize_features(np.array([[3, 6, 9], [4, 8, 12]]))

In [10]:
features

array([[0.6, 0.6, 0.6],
       [0.8, 0.8, 0.8]])

In [11]:
norms

array([ 5., 10., 15.])

## Implementing Coordinate Descent with normalized features

## Effect of L1 penalty

In [12]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

d:\anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
d:\anaconda\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [14]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [16]:
weights = np.array([1, 4, 1])

In [17]:
prediction = predict_output(simple_feature_matrix, weights)

In [18]:
ro = list()
print(weights)
for i in range(0,len(simple_features)+1):
    print(i)
    feature_i = simple_feature_matrix[:,i]
    ro.append( sum((feature_i)*(output - prediction + weights[i]*feature_i) ))
ro

[1 4 1]
0
1
2


[79400300.01452321, 87939470.82325152, 80966698.66623905]

In [20]:
l1_penalty=[1.4e8,1.64e8,1.73e8,1.9e8,2.3e8]
for penalty in l1_penalty:
    if -penalty/2 < ro[2]<penalty/2 and  (ro[1]>penalty/2 or ro[1] < -penalty/2):
        print(penalty)

164000000.0
173000000.0


In [22]:
for penalty in l1_penalty:
    if -penalty/2 <ro[2]<penalty/2 and -penalty/2 < ro[1] < penalty/2:
        print(penalty)

190000000.0
230000000.0


## Single Coordinate Descent Step

In [23]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    prediction = predict_output(feature_matrix, weights)
    feature_i = feature_matrix[:, i]
    ro_i = sum((feature_i)*(output-prediction + weights[i]*feature_i))
    if i == 0:
        new_weight_i = ro_i
    elif ro_i < -l1_penalty/2:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0
    return new_weight_i

In [24]:
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent

In [25]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    while True:
        maxstep = -1
        maxi = 0
        weights = np.zeros(len(initial_weights))
        for i in range(len(initial_weights)):
            weights[i] = lasso_coordinate_descent_step(i,feature_matrix,output,initial_weights,l1_penalty)
            step = abs(weights[i]-initial_weights[i])
            if(step>maxstep):
                maxstep = step
                maxi = i          
        initial_weights[maxi]=weights[maxi]
        #print maxi,maxstep
        if maxstep < tolerance:
            return initial_weights

In [26]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1

In [27]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

d:\anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
d:\anaconda\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [28]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
RSS = sum((predict_output(normalized_simple_feature_matrix,weights)-output)**2)
print(RSS,weights)

1630492383845787.8 [21624988.74465544 63157256.49484932        0.        ]


## Evaluating LASSO fit with more features

In [30]:
train_data = pd.read_csv('kc_house_train_data.csv',dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv',dtype=dtype_dict)
train_data['floors'] = train_data['floors'].astype(float).astype(int)
test_data['floors'] = test_data['floors'].astype(float).astype(int)

In [31]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [32]:
(feature_matrix, output) = get_numpy_data(train_data, all_features, 'price')
normalized_training,norms = normalize_features(feature_matrix)

d:\anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
d:\anaconda\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [33]:
initial_weights = np.zeros(14)
weights1e7 = lasso_cyclical_coordinate_descent(normalized_training, output,
                                            initial_weights, 1e7, 1)
a7 = weights1e7

In [34]:
a7

array([24429589.93019407,        0.        ,        0.        ,
       48389184.67771951,        0.        ,        0.        ,
        3317511.59607863,  7329960.45195357,        0.        ,
              0.        ,        0.        ,        0.        ,
              0.        ,        0.        ])

In [37]:
for i in range(0,len(weights1e7)):
    if weights1e7[i]!=0 and i==0:
        print(0,'constant')
    elif weights1e7[i]!=0:
        print(i,all_features[i-1])

0 constant
3 sqft_living
6 waterfront
7 view


In [39]:
initial_weights = np.zeros(14)
weights1e8 = lasso_cyclical_coordinate_descent(normalized_training, output,
                                            initial_weights, 1e8, 1)

for i in range(0,len(weights1e8)):
    if weights1e8[i]!=0:
        if i == 0:
            print(0,'constant')
        else:
            print(i,all_features[i-1])
a8 = weights1e8

0 constant


In [40]:
initial_weights = np.zeros(14)
weights1e4 = lasso_cyclical_coordinate_descent(normalized_training, output,
                                            initial_weights, 1e4, 5e5)
for i in range(0,len(weights1e4)):
    if weights1e4[i]!=0:
        if i ==0:
            print(0,'constant')
        else:
            print(i,all_features[i-1])
a4 = weights1e4

1 bedrooms
3 sqft_living
4 sqft_lot
5 floors
6 waterfront
7 view
8 condition
9 grade
12 yr_built
13 yr_renovated


## Rescaling learned weights

In [41]:
normalized_weights1e4=a4/norms

normalized_weights1e7=a7/norms

normalized_weights1e8=a8/norms

In [42]:
normalized_weights1e7[3]

161.31749067082316

In [43]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

d:\anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
d:\anaconda\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [44]:
RSS_weight1e4 = sum((predict_output(test_feature_matrix,normalized_weights1e4)-test_output)**2)
RSS_weight1e7 = sum((predict_output(test_feature_matrix,normalized_weights1e7)-test_output)**2)
RSS_weight1e8 = sum((predict_output(test_feature_matrix,normalized_weights1e8)-test_output)**2)

In [45]:
print(RSS_weight1e4)
print(RSS_weight1e7)
print(RSS_weight1e8)

225789014748755.16
275962057997703.1
537166151497322.4
